In [66]:
import sys, os, time
import pycountry

import pandas as pd
import geopandas as gpd

from tqdm.notebook import tqdm
from shapely.geometry import Point

sys.path.append(r"C:\WBG\Work\Code\GOSTrocks\src")
import GOSTrocks.dataMisc as dMisc
from GOSTrocks.misc import tPrint

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
FUA_file = r"C:\WBG\Work\data\URBAN\GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg"
admin_bounds = r"C:\WBG\Work\AdminBounds\Admin0_Polys.shp"

out_folder = r"C:\WBG\Work\MENA_Urban\RESULTS\ACLED"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
inFUA = gpd.read_file(FUA_file)
inAdmin = gpd.read_file(admin_bounds)

inFUA = inFUA.to_crs(4326)
inAdmin = inAdmin.to_crs(4326)

alced_creds = dMisc.get_acled_creds()
acled_email = alced_creds['email']
acled_key = alced_creds['api_key']

In [59]:
# Join the FUA to the admin boundaries to find multi-national cities
inFUA_joined = gpd.sjoin(inFUA, inAdmin, how='inner')

inFUA_joined['MultiISO'] = inFUA_joined.duplicated(subset='eFUA_ID', keep=False)
# Create two datasets - simple FUAs are contained within one country, complicated FUAs are multi-national
simple_FUA = inFUA_joined.loc[inFUA_joined['MultiISO'] == False]
complicated_FUA = inFUA_joined.loc[inFUA_joined['MultiISO'] == True]
complicated_FUA.drop_duplicates(subset='eFUA_ID', keep='first', inplace=True)

In [64]:
simple_FUA.head()

,eFUA_ID,UC_num,UC_IDs,eFUA_name,Commuting,Cntry_ISO,Cntry_name,FUA_area,UC_area,FUA_p_2015,...,WB_ADM0_NA,Shape_Leng,Shape_Area,ISO3,UN_m49,Region,incomeG,lendingC,FID_100,MultiISO
0,1.0,1.0,5959,Herat,1.0,AFG,Afghanistan,139.0,60.0,888811.175807,...,Afghanistan,7.132529e+06,641837.888397,AFG,4,South Asia,Low income,IDA,0,False
1,192.0,1.0,5964,Guzarah,1.0,AFG,Afghanistan,32.0,16.0,169489.573231,...,Afghanistan,7.132529e+06,641837.888397,AFG,4,South Asia,Low income,IDA,0,False
2,354.0,1.0,5968,Shindand,0.0,AFG,Afghanistan,12.0,12.0,147553.403870,...,Afghanistan,7.132529e+06,641837.888397,AFG,4,South Asia,Low income,IDA,0,False
3,505.0,1.0,5970,Qala i Naw,0.0,AFG,Afghanistan,3.0,3.0,79809.722656,...,Afghanistan,7.132529e+06,641837.888397,AFG,4,South Asia,Low income,IDA,0,False
4,648.0,1.0,5973,Farah,1.0,AFG,Afghanistan,32.0,16.0,131508.797060,...,Afghanistan,7.132529e+06,641837.888397,AFG,4,South Asia,Low income,IDA,0,False


In [101]:
# For simple FUAs, we will query the ACLED database with the ISO codes
for iso3, curData in simple_FUA.groupby('Cntry_ISO'):
    out_file = os.path.join(out_folder, '%s_ACLED.csv' % iso3)
    if not os.path.exists(out_file):
        try:
            country_coding = pycountry.countries.get(alpha_3=iso3).numeric
        except:
            country_coding = 0 #For Kosovo and other potentially unrecognized states
        acled_data = dMisc.acled_search(acled_key, acled_email, iso3 = country_coding,
                                        fields = ['longitude','latitude','event_type','geo_precision'])
        if acled_data.shape[0] > 0:
            acled_geom = [Point(x, y) for x, y in zip(acled_data['longitude'], acled_data['latitude'])]
            acled_geo = gpd.GeoDataFrame(acled_data, geometry = acled_geom, crs = 4326)
            country_res = []
            for idx, row in tqdm(curData.iterrows(), total=curData.shape[0], desc='Processing ACLED data for %s' % iso3):
                row_df = gpd.GeoDataFrame(row.to_frame().T, geometry='geometry', crs=4326)
                row_df.set_index('eFUA_ID', inplace=True)
                sel_acled = gpd.sjoin(acled_geo, row_df, how='inner')
                if sel_acled.shape[0] > 0:                        
                    # Summarize conflict type and geoprecision
                    acled_summary = sel_acled.groupby(['event_type', 'geo_precision']).size().reset_index(name='counts')
                    acled_summary['Label'] = acled_summary['event_type'] + '_' + acled_summary['geo_precision'].astype(str)
                    acled_summary = acled_summary.loc[:,['Label', 'counts']].T
                    acled_summary.columns = acled_summary.loc['Label']
                    acled_summary = acled_summary.drop('Label')
                    acled_summary['eFUA_ID'] = row['eFUA_ID']
                    country_res.append(acled_summary)
            if len(country_res) > 0:
                country_total = pd.concat(country_res)
                country_total.to_csv(out_file)


c:\wbg\Anaconda3\envs\s2s\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Processing ACLED data for XKO:   0%|          | 0/7 [00:00<?, ?it/s]

c:\wbg\Anaconda3\envs\s2s\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Processing ACLED data for XNC:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: No objects to concatenate

In [99]:
pycountry.countries.get(name=curData['Cntry_name'].iloc[0])

In [100]:
curData['Cntry_name'].iloc[0]

'Kosovo'

Processing ACLED data for AFG:   0%|          | 0/28 [00:00<?, ?it/s]

In [92]:
pd.concat(country_res)            

Label,Battles_1,Battles_2,Battles_3,Explosions/Remote violence_1,Explosions/Remote violence_2,Explosions/Remote violence_3,Protests_1,Protests_2,Protests_3,Riots_1,Riots_3,Strategic developments_1,Strategic developments_2,Strategic developments_3,Violence against civilians_1,Violence against civilians_2,Violence against civilians_3,eFUA_ID,Riots_2
counts,158,54,101,49,15,17,47,1,28,5,1,21,2,29,96,20,26,1.0,NaN
counts,1,72,1,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,10,NaN,192.0,NaN
counts,12,336,NaN,3,120,NaN,NaN,1,NaN,NaN,NaN,NaN,9,NaN,1,21,NaN,354.0,NaN
counts,60,39,94,27,11,51,5,1,6,1,1,5,1,13,12,4,2,505.0,NaN
counts,258,97,113,95,33,51,6,NaN,6,1,NaN,12,1,12,35,11,13,648.0,NaN
counts,24,12,46,5,1,23,9,NaN,9,1,NaN,4,2,9,6,7,13,784.0,NaN
counts,43,192,167,18,61,44,23,1,9,2,NaN,12,4,13,26,12,13,916.0,NaN
counts,56,28,70,30,8,9,4,1,7,1,NaN,2,3,8,18,4,11,1040.0,NaN
counts,67,24,47,38,7,12,5,NaN,2,NaN,NaN,9,NaN,11,12,3,7,1154.0,NaN
counts,404,90,129,221,39,57,25,1,21,2,NaN,18,7,24,54,8,7,1263.0,NaN


In [42]:
all_res = {}
for idx, row in tqdm(inFUA.iterrows(), total = inFUA.shape[0]):
    acled_data = dMisc.acled_search(acled_key, acled_email, row.geometry.bounds,
                                    fields = ['longitude','latitude','event_type','geo_precision'])
    if acled_data.shape[0] > 0:
        acled_geom = [Point(x, y) for x, y in zip(acled_data['longitude'], acled_data['latitude'])]
        acled_geo = gpd.GeoDataFrame(acled_data, geometry = acled_geom, crs = 4326)
        acled_geo = acled_geo.loc[acled_geo.within(row.geometry)]
        if acled_geo.shape[0] > 0:                
            # Tabulate results
            acled_summary = acled_geo.groupby(['event_type', 'geo_precision']).size().reset_index(name='counts')
            acled_summary['Label'] = acled_summary['event_type'] + '_' + acled_summary['geo_precision'].astype(str)
            acled_summary = acled_summary.loc[:,['Label', 'counts']].T
            acled_summary.columns = acled_summary.loc['Label']
            acled_summary = acled_summary.drop('Label')
            acled_summary['eFUA_ID'] = row['eFUA_ID']
            all_res[idx] = acled_summary    
    


  0%|          | 0/9031 [00:00<?, ?it/s]

c:\wbg\Anaconda3\envs\s2s\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.acleddata.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s\Lib\site-packages\urlli

KeyboardInterrupt: 

In [54]:
all_dfs = [item for key, item in all_res.items()]
pd.concat(all_dfs)

Label,Battles_1,Battles_2,Battles_3,Explosions/Remote violence_1,Explosions/Remote violence_2,Explosions/Remote violence_3,Protests_1,Protests_2,Protests_3,Riots_1,Riots_3,Strategic developments_1,Strategic developments_2,Strategic developments_3,Violence against civilians_1,Violence against civilians_2,Violence against civilians_3,eFUA_ID,Riots_2
counts,158,54,101,49,15,17,47,1,28,5,1,21,2,29,96,20,26,1.0,NaN
counts,1,72,1,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,10,NaN,192.0,NaN
counts,12,336,NaN,3,120,NaN,NaN,1,NaN,NaN,NaN,NaN,9,NaN,1,21,NaN,354.0,NaN
counts,60,39,94,27,11,51,5,1,6,1,1,5,1,13,12,4,2,505.0,NaN
counts,258,97,113,95,33,51,6,NaN,6,1,NaN,12,1,12,35,11,13,648.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
counts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,1,NaN,NaN,1910.0,NaN
counts,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,4,1,NaN,1989.0,NaN
counts,3,NaN,NaN,1,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,2064.0,NaN
counts,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,1,NaN,NaN,2213.0,NaN


In [49]:
pd.DataFrame(all_res

Label,Battles_1,Battles_2,Battles_3,Explosions/Remote violence_2,Strategic developments_2,Violence against civilians_2,eFUA_ID
counts,1,72,1,5,5,10,192.0
